In [3]:
from multiprocessing import Process, Queue

# Camera Agent - captures votes and sends to Counting Agent
def camera_agent(queue):
    votes = ["Candidate_A", "Candidate_B", "Candidate_A", "Candidate_C" , "Candidate_C", "Candidate_C"]
    for vote in votes:
        print(f"Camera agent captured vote: {vote}")
        queue.put(vote)  # Sending vote to Counting Agent

# Counting Agent - receives votes from Camera Agent and counts them
def counting_agent(queue):
    vote_counts = {"Candidate_A": 0, "Candidate_B": 0, "Candidate_C": 0}
    while True:
        vote = queue.get()  # Receive vote from Camera Agent
        if vote == "DONE":  # Use a sentinel value to end the loop
            break
        vote_counts[vote] += 1
    print(f"Counting Agent tallied votes: {vote_counts}")

def main():
    # Create a Queue to simulate communication between Camera and Counting Agents
    vote_queue = Queue()

    # Create processes for Camera Agent and Counting Agent
    camera = Process(target=camera_agent, args=(vote_queue,))
    counter = Process(target=counting_agent, args=(vote_queue,))

    # Start both processes
    camera.start()
    counter.start()

    # Wait for both processes to finish
    camera.join()
    vote_queue.put("DONE")  # Stop Counting Agent with a sentinel value
    counter.join()

if __name__ == "__main__":
    main()


Camera agent captured vote: Candidate_A
Camera agent captured vote: Candidate_B
Camera agent captured vote: Candidate_A
Camera agent captured vote: Candidate_C
Camera agent captured vote: Candidate_C
Camera agent captured vote: Candidate_C
Counting Agent tallied votes: {'Candidate_A': 2, 'Candidate_B': 1, 'Candidate_C': 3}


In [8]:
from multiprocessing import Process, Queue

# Function to simulate vote detection
def detect_vote(ballot_paper):
    """
    Simulate detecting votes on a ballot paper.
    Check the rightmost column for votes.
    Returns a list of detected votes with their row and column indices.
    """
    detected_votes = []
    invalid_votes = 0

    for row_index, row in enumerate(ballot_paper):
        if row[0] == 1:  # Check the rightmost column (column index 0)
            detected_votes.append((row_index, 0))  # Record the row and column index
        else:
            invalid_votes += 1

    return detected_votes, invalid_votes

# Function to send detected votes to the counting agent
def camera_agent(queue):
    # Simulated ballot paper (3 columns, 6 rows)
    # Example: 1 in column 0 indicates a vote, 0 means no vote
    ballot_paper = [
        [1, 0, 0],  # Row 0, vote detected in column 0
        [1, 0, 0],  # Row 1, vote detected in column 0
        [0, 0, 0],  # Row 2, no vote detected
        [1, 0, 0],  # Row 3, vote detected in column 0
        [0, 0, 0],  # Row 4, no vote detected
        [0, 0, 0],  # Row 5, no vote detected
    ]

    detected_votes, invalid_votes = detect_vote(ballot_paper)
    print(f"Camera agent detected votes: {detected_votes}")
    print(f"Invalid votes: {invalid_votes}")

    queue.put((detected_votes, invalid_votes))  # Send to Counting Agent

# Function to count votes based on row index
def count_votes(detected_votes, invalid_votes):
    """
    Count votes for each party based on the row index.
    """
    party_votes = {"Party_01": 0, "Party_02": 0, "Party_03": 0, "Party_04": 0, "Party_05": 0, "Party_06": 0}
    for row_index, col_index in detected_votes:
        # Ensure party name is in the format 'Party_01', 'Party_02', etc.
        party_key = f"Party_{row_index + 1:02d}"  # Pad row index with leading zero for 'Party_01'
        party_votes[party_key] += 1

    return party_votes, invalid_votes

# Counting Agent - receives votes and tallies them
def counting_agent(queue):
    detected_votes, invalid_votes = queue.get()  # Receive votes from Camera Agent
    party_votes, invalid_count = count_votes(detected_votes, invalid_votes)
    print(f"Counting Agent tallied votes: {party_votes}")
    print(f"Invalid votes count: {invalid_count}")

# Main function to run the Camera Agent and Counting Agent
def main():
    queue = Queue()

    # Create processes for Camera Agent and Counting Agent
    camera = Process(target=camera_agent, args=(queue,))
    counter = Process(target=counting_agent, args=(queue,))

    # Start both processes
    camera.start()
    counter.start()

    # Wait for both processes to finish
    camera.join()
    counter.join()

if __name__ == "__main__":
    main()


Camera agent detected votes: [(0, 0), (1, 0), (3, 0)]
Invalid votes: 3
Counting Agent tallied votes: {'Party_01': 1, 'Party_02': 1, 'Party_03': 0, 'Party_04': 1, 'Party_05': 0, 'Party_06': 0}
Invalid votes count: 3


In [13]:
from multiprocessing import Process, Queue

# Function to simulate vote detection (with only one vote per ballot paper)
def detect_vote(ballot_paper):
    """
    Simulate detecting votes on a ballot paper.
    Check the rightmost column (column index 0) for votes.
    If there is a valid vote, return the row and column index.
    If no vote is detected, return 'invalid'.
    """
    detected_vote = None

    # Check if a vote is detected in the rightmost column (column 0)
    for row_index, row in enumerate(ballot_paper):
        if row[0] == 1:  # Vote detected in the rightmost column
            if detected_vote is None:  # If no previous vote was detected
                detected_vote = (row_index, 0)  # Record the row and column index of the vote
            else:
                # If a vote is already detected, mark as invalid
                detected_vote = 'invalid'
                break

    # If no vote was detected, mark as invalid
    if detected_vote is None:
        detected_vote = 'invalid'

    return detected_vote

# Function to send detected votes to the counting agent
def camera_agent(queue):
    # Simulated ballot paper (3 columns, 6 rows)
    # Example: 1 in column 0 indicates a vote, 0 means no vote
    ballot_paper = [
        [1, 0, 0],  # Row 0, vote detected in column 0 (valid)
        [0, 0, 0],  # Row 1, no vote detected
        [0, 0, 0],  # Row 2, no vote detected
        [0, 0, 0],  # Row 3, no vote detected
        [0, 0, 0],  # Row 4, no vote detected
        [0, 0, 0],  # Row 5, no vote detected
    ]

    detected_vote = detect_vote(ballot_paper)
    print(f"Camera agent detected vote: {detected_vote}")

    queue.put(detected_vote)  # Send detected vote (valid or invalid) to Counting Agent

# Function to count votes based on row index
def count_votes(detected_vote):
    """
    Count votes for each party based on the row index.
    If the vote is invalid, return invalid vote count.
    """
    party_votes = {"Party_01": 0, "Party_02": 0, "Party_03": 0, "Party_04": 0, "Party_05": 0, "Party_06": 0}
    invalid_votes = 0

    if detected_vote == 'invalid':
        invalid_votes = 1
    else:
        # If a valid vote is detected, map it to the corresponding party
        row_index, col_index = detected_vote
        party_key = f"Party_{row_index + 1:02d}"  # Map row index to party
        party_votes[party_key] += 1

    return party_votes, invalid_votes

# Counting Agent - receives votes and tallies them
def counting_agent(queue):
    detected_vote = queue.get()  # Receive detected vote from Camera Agent
    party_votes, invalid_count = count_votes(detected_vote)
    print(f"Counting Agent tallied votes: {party_votes}")
    print(f"Invalid votes count: {invalid_count}")

# Main function to run the Camera Agent and Counting Agent
def main():
    queue = Queue()

    # Create processes for Camera Agent and Counting Agent
    camera = Process(target=camera_agent, args=(queue,))
    counter = Process(target=counting_agent, args=(queue,))

    # Start both processes
    camera.start()
    counter.start()

    # Wait for both processes to finish
    camera.join()
    counter.join()

if __name__ == "__main__":
    main()


Camera agent detected vote: (0, 0)
Counting Agent tallied votes: {'Party_01': 1, 'Party_02': 0, 'Party_03': 0, 'Party_04': 0, 'Party_05': 0, 'Party_06': 0}
Invalid votes count: 0


In [15]:
from multiprocessing import Process, Queue

# Function to simulate vote detection (Detects any vote in the rightmost column)
def detect_vote(ballot_paper):
    """
    Simulate detecting a vote in the rightmost column (or any column).
    Return the row and column index if a vote is detected, otherwise 'invalid'.
    """
    detected_vote = None

    # Check if a vote is detected in the rightmost column (column 0)
    for row_index, row in enumerate(ballot_paper):
        if row[0] == 1:  # Vote detected in the rightmost column
            if detected_vote is None:  # If no previous vote was detected
                detected_vote = (row_index, 0)  # Record the row and column index of the vote
            else:
                # If a vote is already detected, mark as invalid
                detected_vote = 'invalid'
                break

    # If no vote was detected, mark as invalid
    if detected_vote is None:
        detected_vote = 'invalid'

    return detected_vote

# Function to send detected votes to the counting agent
def camera_agent(queue, ballot_paper):
    """
    Camera agent detects votes and sends the row and column index to the counting agent.
    """
    detected_vote = detect_vote(ballot_paper)
    print(f"Camera agent detected vote: {detected_vote}")

    # Send detected vote (row and column index or 'invalid') to Counting Agent
    queue.put(detected_vote)

# Function to count votes based on received row index
def count_votes(detected_vote, num_parties):
    """
    Count votes for each party based on the row index received from Camera Agent.
    If the vote is invalid, return invalid vote count.
    """
    # Dynamically create the party votes dictionary based on the number of parties
    party_votes = {f"Party_{i+1:02d}": 0 for i in range(num_parties)}  # Create dynamic party structure
    invalid_votes = 0

    if detected_vote == 'invalid':
        invalid_votes = 1
    else:
        # If a valid vote is detected, map it to the corresponding party
        row_index, col_index = detected_vote
        if 0 <= row_index < num_parties:  # Ensure the vote corresponds to a valid party
            party_key = f"Party_{row_index + 1:02d}"  # Map row index to party
            party_votes[party_key] += 1
        else:
            # If the row index is out of bounds, mark as invalid
            invalid_votes = 1

    return party_votes, invalid_votes

# Counting Agent - receives votes and tallies them
def counting_agent(queue, num_parties):
    """
    Counting agent receives the vote (row and column index or 'invalid') from the camera agent
    and counts votes for each party.
    """
    detected_vote = queue.get()  # Receive detected vote from Camera Agent
    party_votes, invalid_count = count_votes(detected_vote, num_parties)
    print(f"Counting Agent tallied votes: {party_votes}")
    print(f"Invalid votes count: {invalid_count}")

# Main function to run the Camera Agent and Counting Agent
def main():
    # Define the number of parties (for flexibility)
    num_parties = 6  # Example: 6 political parties

    # Simulated ballot paper (3 columns, 6 rows)
    # Example: 1 in column 0 indicates a vote, 0 means no vote
    ballot_paper = [
        [1, 0, 0],  # Row 0, vote detected in column 0 (valid)
        [0, 0, 0],  # Row 1, no vote detected
        [0, 0, 0],  # Row 2, no vote detected
        [0, 0, 0],  # Row 3, no vote detected
        [0, 0, 0],  # Row 4, no vote detected
        [0, 0, 0],  # Row 5, no vote detected
    ]

    queue = Queue()

    # Create processes for Camera Agent and Counting Agent
    camera = Process(target=camera_agent, args=(queue, ballot_paper))
    counter = Process(target=counting_agent, args=(queue, num_parties))

    # Start both processes
    camera.start()
    counter.start()

    # Wait for both processes to finish
    camera.join()
    counter.join()

if __name__ == "__main__":
    main()


Camera agent detected vote: (0, 0)
Counting Agent tallied votes: {'Party_01': 1, 'Party_02': 0, 'Party_03': 0, 'Party_04': 0, 'Party_05': 0, 'Party_06': 0}
Invalid votes count: 0
